In [1]:
from pathlib import Path
import pandas as pd

In [2]:
import sys
sys.path.append('../../src')
from DataLoader import DataLoader

In [3]:
fileLoader = DataLoader("../config.yaml")
df_opalmeta = fileLoader.load_data("files.df_opalmeta")
df_opalmeta.head(3)

/media/sz/Data/Connected_Lecturers/Opal_crosslab/raw/OPAL_files_attrib.p


,opal:filename,opal:oer_permalink,opal:license,opal:author,opal:title,opal:comment,opal:language,opal:publicationMonth,opal:publicationYear,opal:revisedAuthor,pipe:ID,pipe:file_type
0,SC_Hardware.png,https://bildungsportal.sachsen.de/opal/oer/6TI...,CC BY 4.0 Int.,Daniela Dobeleit,Hardware zum Styroporschneiden,,,,,[Vorname='Daniela' Familienname='Dobeleit' Tit...,6TIlFGfTxXkw,png
1,beleg.pdf,https://bildungsportal.sachsen.de/opal/oer/8I6...,CC BY-SA 4.0 Int.,,,,,,,,8I6sM5zapD60,pdf
2,Bereich III.pdf,https://bildungsportal.sachsen.de/opal/oer/8ZI...,CC BY-NC-ND 4.0 Int.,,,,,,,,8ZICOHBmAHyQ,pdf


In [4]:
df_filesmeta = fileLoader.load_data("files.df_filesmeta")
df_filesmeta.head(3)

/media/sz/Data/Connected_Lecturers/Opal_crosslab/raw/OPAL_files_meta.p


,pipe:ID,pipe:file_type,file:author,file:keywords,file:subject,file:title,file:created,file:modified,file:language,file:revisedAuthor
0,8I6sM5zapD60,pdf,,,,,2019-01-11 17:12:31+00:00,2019-01-11 17:12:31+00:00,NaN,
1,8ZICOHBmAHyQ,pdf,,,,,2019-10-25 12:05:54+00:00,2019-10-25 12:05:54+00:00,NaN,
2,8Lfz8SAKa6k0,pdf,,,,,2024-12-20 07:48:12+00:00,NaT,NaN,


In [5]:
df_aimeta = fileLoader.load_data("files.df_aimeta")
df_aimeta.head(3)

/media/sz/Data/Connected_Lecturers/Opal_crosslab/raw/OPAL_ai_meta.p


,pipe:ID,pipe:file_type,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:keywords_gen,ai:keywords_dnb,ai:dewey
0,8I6sM5zapD60,pdf,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Photogrammetrie, digitale Entzerrung, Resampli...","Photogrammetrie, digitale Entzerrung, Resampli...","Photogrammetrie, Bildverarbeitung, Digitale En...",
1,8ZICOHBmAHyQ,pdf,"(Leerstring, da kein Autor im Dokument erwähnt...",[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...","Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...","Tragwerke, Statik, Mechanik, Festigkeitslehre,...",
2,8Lfz8SAKa6k0,pdf,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...","Künstliche Intelligenz, Automatisierung, Indus...","Künstliche Intelligenz, Automatisierung, Indus...",


## Let's start with authors names ...

In [6]:
def extract_first_three_lastnames(name_list):
    """
    Extrahiert die ersten drei Nachnamen aus der ai:revisedAuthor Spalte.
    
    Args:
        name_list (list): Liste von Name-Objekten aus der ai:revisedAuthor Spalte
        
    Returns:
        str: String mit den ersten drei Nachnamen, getrennt durch Kommata
    """
    # Prüfe ob die Liste leer oder None ist
    if not name_list or len(name_list) == 0:
        return ""
    
    # Extrahiere die Familiennamen aus den ersten drei Name-Objekten
    lastnames = []
    for i, name_obj in enumerate(name_list[:3]):  # Nur die ersten drei
        if hasattr(name_obj, 'Familienname'):
            lastname = name_obj.Familienname
            if len(lastname) > 1:
                if lastname and lastname.strip():  # Prüfe auf leere Strings
                    lastnames.append(lastname.strip())
    
    # Verbinde die Nachnamen mit Kommata zu einem String
    return ", ".join(lastnames)

In [7]:
df_aimeta['ai:revisedAuthorstr'] = df_aimeta['ai:revisedAuthor'].apply(extract_first_three_lastnames)
df_filesmeta['file:revisedAuthorstr'] = df_filesmeta['file:revisedAuthor'].apply(extract_first_three_lastnames)
df_opalmeta['opal:revisedAuthorstr'] = df_opalmeta['opal:revisedAuthor'].apply(extract_first_three_lastnames)

In [8]:
# merge all three dataframe by pipe:ID. Just keep 'ai:revisedAuthorstr', 'file:revisedAuthorstr', 'opal:revisedAuthorstr'
df_nodes = df_aimeta[['pipe:ID', 'ai:revisedAuthorstr', 'ai:affilation',
                      'ai:title', 'pipe:file_type', 'ai:type', 'ai:keywords_dnb'		]].merge(
    df_filesmeta[['pipe:ID', 'file:revisedAuthorstr']],
    on='pipe:ID',
    how='left'
).merge(
    df_opalmeta[['pipe:ID', 'opal:revisedAuthorstr','opal:oer_permalink']],
    on='pipe:ID',
    how='left'
)

In [9]:
#  replace all NaN strings with empty strings
df_nodes = df_nodes.fillna('')

df_nodes['Authors'] = "nicht bekannt"

# Wenn nur eine Spalte Werte hat, verwende diese (aber nur wenn sie nicht leer ist!)
df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] == '') & 
                      (df_nodes['file:revisedAuthorstr'] == '') &
                      (df_nodes['opal:revisedAuthorstr'] != ''), 'Authors'] = df_nodes['opal:revisedAuthorstr']

df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] == '') & 
                      (df_nodes['opal:revisedAuthorstr'] == '') &
                      (df_nodes['file:revisedAuthorstr'] != ''), 'Authors'] = df_nodes['file:revisedAuthorstr']

df_nodes.loc[(df_nodes['file:revisedAuthorstr'] == '') & 
                      (df_nodes['opal:revisedAuthorstr'] == '') &
                      (df_nodes['ai:revisedAuthorstr'] != ''), 'Authors'] = df_nodes['ai:revisedAuthorstr']

# Wenn zwei Spalten Werte haben und gleich sind
df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] != '') & 
                      (df_nodes['file:revisedAuthorstr'] != '') &
                      (df_nodes['ai:revisedAuthorstr'] == df_nodes['file:revisedAuthorstr']), 'Authors'] = df_nodes['ai:revisedAuthorstr']

df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] != '') & 
                      (df_nodes['opal:revisedAuthorstr'] != '') &
                      (df_nodes['ai:revisedAuthorstr'] == df_nodes['opal:revisedAuthorstr']), 'Authors'] = df_nodes['ai:revisedAuthorstr']

df_nodes.loc[(df_nodes['file:revisedAuthorstr'] != '') & 
                      (df_nodes['opal:revisedAuthorstr'] != '') &
                      (df_nodes['file:revisedAuthorstr'] == df_nodes['opal:revisedAuthorstr']), 'Authors'] = df_nodes['file:revisedAuthorstr']   

# Wenn zwei oder mehr Spalten unterschiedliche Werte haben, bevorzuge ai:revisedAuthorstr
df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] != '') & 
                      (df_nodes['file:revisedAuthorstr'] != '') &
                      (df_nodes['ai:revisedAuthorstr'] != df_nodes['file:revisedAuthorstr']), 'Authors'] = df_nodes['ai:revisedAuthorstr']

df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] != '') & 
                      (df_nodes['opal:revisedAuthorstr'] != '') &
                      (df_nodes['ai:revisedAuthorstr'] != df_nodes['opal:revisedAuthorstr']), 'Authors'] = df_nodes['ai:revisedAuthorstr']

# Aber wenn ai leer ist und file/opal unterschiedlich sind, bevorzuge file
df_nodes.loc[(df_nodes['ai:revisedAuthorstr'] == '') & 
                      (df_nodes['file:revisedAuthorstr'] != '') & 
                      (df_nodes['opal:revisedAuthorstr'] != '') &
                      (df_nodes['file:revisedAuthorstr'] != df_nodes['opal:revisedAuthorstr']), 'Authors'] = df_nodes['file:revisedAuthorstr']

In [10]:
def make_link(url, link):
    return f'<a href="{url}">{link}</a><br>'

df_nodes['Link'] = df_nodes.apply(lambda x: make_link(x['opal:oer_permalink'], "Link"), axis=1)

In [11]:
df_nodes.columns

Index(['pipe:ID', 'ai:revisedAuthorstr', 'ai:affilation', 'ai:title',
       'pipe:file_type', 'ai:type', 'ai:keywords_dnb', 'file:revisedAuthorstr',
       'opal:revisedAuthorstr', 'opal:oer_permalink', 'Authors', 'Link'],
      dtype='object')

In [12]:
df_nodes.drop(columns=['ai:revisedAuthorstr', 
                       'file:revisedAuthorstr',
                       'opal:revisedAuthorstr',
                       'opal:oer_permalink'], inplace=True)


df_nodes.rename(columns={
    'pipe:ID': 'ID',
    'ai:affilation': 'Affiliation',
    'ai:title': 'Title',
    'pipe:file_type': 'File Type',
    'ai:type': 'Type',
    'ai:keywords_dnb': 'Keywords DNB',
}, inplace=True)   


In [13]:
df_nodes

,ID,Affiliation,Title,File Type,Type,Keywords DNB,Authors,Link
0,8I6sM5zapD60,,Übung 3 Photogrammetrie,pdf,Aufgabenblatt,"Photogrammetrie, Bildverarbeitung, Digitale En...","Gerhold, Beyer","<a href=""https://bildungsportal.sachsen.de/opa..."
1,8ZICOHBmAHyQ,,3.1 Grundlagen der Tragwerke,pdf,Skript,"Tragwerke, Statik, Mechanik, Festigkeitslehre,...",nicht bekannt,"<a href=""https://bildungsportal.sachsen.de/opa..."
2,8Lfz8SAKa6k0,,Gedanken und Notizen zu unserem Mini-KI-Worksh...,pdf,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...",Ringel,"<a href=""https://bildungsportal.sachsen.de/opa..."
3,3ztCv-WpxJ4U,,Allgemeine Anforderungen,pdf,Aufgabenblatt,"Wissenschaftliches Arbeiten, akademische Schri...",Engemaier,"<a href=""https://bildungsportal.sachsen.de/opa..."
4,6mOhjfscZK2A,TU Dresden,"Vorlesung Technische Mechanik I - AGBF, TU Dre...",pdf,Vorlesungsfolien,"Technische Mechanik, Starrer Körper, Kraft, Mo...",nicht bekannt,"<a href=""https://bildungsportal.sachsen.de/opa..."
...,...,...,...,...,...,...,...,...
4543,8Qx2WQd_ANdI,,Grundlagen der Linguistik und Sprachgeschichte...,pdf,Aufgabenblatt,"Linguistik, Deutsche Sprache, Dialektologie, V...","Niebaum, Macha","<a href=""https://bildungsportal.sachsen.de/opa..."
4544,7rwytM7w-8HE,,"Emotionsmanagement (Fähigkeit, eigene Emotione...",pdf,Skript,"Intelligenz, Kreativität, Emotionsmanagement, ...",nicht bekannt,"<a href=""https://bildungsportal.sachsen.de/opa..."
4545,8MBQRpA2XSQU,,Chaotisches Verhalten,pdf,Skript,"Chaostheorie, Deterministisches Chaos, Nichtli...",nicht bekannt,"<a href=""https://bildungsportal.sachsen.de/opa..."
4546,10MPMUvFFT_H0,Hochschule für Technik und Wirtschaft Dresden,Übungsblatt 9 Aufgaben mit Lösungshilfe,pdf,Aufgabenblatt,"Mathematik, Geometrie, Lineare Algebra, Drehun...","Hamann, Meinhold","<a href=""https://bildungsportal.sachsen.de/opa..."


In [ ]:
processed_data_folder = fileLoader.config_manager.get("folder_structure.processed_data_folder")
df_nodes.to_html(Path(processed_data_folder)/'nodes.html', render_links=True, escape=False, index=False) 